In [258]:
import pandas as pd
import math
import os
import numpy as np

In [259]:
import warnings
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

# Read in Soil's table from ArcGIS and pull out the soil properties we need for equations

In [260]:
#read in Soil's table from ArcGIS 
df = pd.read_csv('GSSHA_30m_Soils_Clip.csv')

In [261]:
#select the columns we care about
df = df[['MUSYM','MUKEY','MUName','SurfText','Sand','Clay','Ksatcmhr','Db3rdbar','OrgMatter', 'WC3rdbar', 'WC15Bar']].copy()

#rename columns so that we know the units of these columns (the rest are % and need to be converted to ratio)
df.rename(columns = {'Db3rdbar':'bulkdens_g/cm3', 'WC3rdbar' : '3rdBar_FieldCap', 'WC15Bar' : '15Bar_WiltingPt'}, inplace = True)

In [262]:
#convert str to upper case
df['SurfText'] = df['SurfText'].str.upper()

#conver percentagee columns to ratios
df['Sand'] = df['Sand'] / 100
df['Clay'] = df['Clay'] / 100
df['OrgMatter'] = df['OrgMatter'] / 100
df['3rdBar_FieldCap'] = df['3rdBar_FieldCap'] / 100
df['15Bar_WiltingPt'] = df['15Bar_WiltingPt'] / 100

According to Hanalei SWAT the Hanalei stream channel has a Ksat value of 9.4628  
Stream beds are a high energy depositional environment, so we can assume the
stream channel to be composed of mostly sand or gravel.
Based on observations of other hawaii streams, it is probably a stony land
the porosity, field capacity, and wilting point of stony land is given in
Safeeq 2011. this will be used as the starting point for FC and WP

Riverwash is also going to be the same or very similar to the water...

According to Safeeq and Fares 2011: 
rock outcrop has a fc = 0.42 and wp = 0.34
mucky peat has a fc = 0.52 and wp = 0.36
stony land has a fc = 0.46 and wp = 0.33 aka (riverwash and water)

In [263]:
for i in range(0, len(df)):
    if df['MUName'][i] == 'Rock outcrop':
        df['3rdBar_FieldCap'][i] = 0.42
        df['15Bar_WiltingPt'][i] = 0.34
    if df['MUName'][i] == 'Marsh':
        df['3rdBar_FieldCap'][i] = 0.52
        df['15Bar_WiltingPt'][i] = 0.36
    if df['MUName'][i] == 'Water > 40 acres':
        df['Ksatcmhr'][i] = 9.4628
        df['SurfText'][i] = 'SAND'
        df['3rdBar_FieldCap'][i] = 0.46
        df['15Bar_WiltingPt'][i] = 0.33
    if df['MUName'][i] == 'Riverwash':
        df['3rdBar_FieldCap'][i] = 0.46
        df['15Bar_WiltingPt'][i] = 0.33

In [264]:
#there are a few duplicates of soils in this list... to make things easier and decrease volume lets only look at unique soils
df_nodup = df.drop_duplicates().reset_index(drop = True)

In [265]:
#what values will be a problem....?

print('Missing FC and WP for Riverwash')
print('Missing bulk density, %Sand, %Clay, FC, WP, from Marsh')
print('Missing everything from Water')

Missing FC and WP for Riverwash
Missing bulk density, %Sand, %Clay, FC, WP, from Marsh
Missing everything from Water


# Determine missing bulk density values

In [266]:
#Using the mineral bulk density chart given in Maidment 1993, determine mineral bulk density


def Soil_Bulk_Dens(Organic_Matter, Mineral_Bulk_Density):
    
    AOMBD = 0.224 #average organic matter bulk density
    OM = Organic_Matter*100 #%Organic matter
    MBD = Mineral_Bulk_Density #Mineral bulk density determined by triangle sediment
    
    Soil_Bulk_Density = 100 / ( (OM/AOMBD) +  ((100-OM)/MBD) )
    
    return Soil_Bulk_Density

In [267]:
#variables needed for equation
#97.9% Sand and 0.005% clay for Riverwash
#determined that mineral bulk density = 1.53


Riverwash_OM = 0.0007
Riverwash_MBD = 1.53


#calculate soil bulk density
Riverwash_Soil_BD = round(Soil_Bulk_Dens(Riverwash_OM, Riverwash_MBD),2)


#update bulk density for River (6 is the index!!!!!!!!!!! make sure this shit doesn't fuck u up)
df_nodup['bulkdens_g/cm3'][6] = Riverwash_Soil_BD


# Equations for wierd soil parameters

In [268]:
#create empty columns for variables wanted
df_nodup['Particle_Density_McBride']  = np.nan
df_nodup['Porosity']  = np.nan
df_nodup['Pore_size_dsitribution']  = np.nan
df_nodup['residual_water_saturation']  = np.nan
df_nodup['wetting_front_suction']  = np.nan

Schjonning 2017 shows some equations that relate particle density to soil organic matter and clay using the follow function instead of claiming the value to be 2.65 like most articles do. The equation given McBride 2011 was used.

In [269]:
def particle_density_McBride(soil_organic_matter):
     SOM = soil_organic_matter
    #Equation given by McBride 1991 to account for organic matter when determining particle density
     PD = 2.646 - 2.8*SOM
    
     return PD

In [270]:
for i in range(0, len(df_nodup)):
    OM = df_nodup['OrgMatter'][i]
    PD = particle_density_McBride(OM)
    df_nodup['Particle_Density_McBride'][i] = PD

The equation used for porosity was determined by Maidment 1993 and Loing2017, where Loing used it for calculating the porosity of Hawaii soils

In [271]:
def porosity(Bulk_density,Particle_density):
    porosity = 1- (BD/PD)
    return porosity

In [272]:
for i in range(0, len(df_nodup)):
    BD = df_nodup['bulkdens_g/cm3'][i]
    PD = df_nodup['Particle_Density_McBride'][i]
    #PD = 2.65
    Porosity = porosity(BD, PD)
    df_nodup['Porosity'][i] = Porosity

According to Safeeq 2011, the porosity for rock outcrops is 0.666 and the porosity of Mucky Peat is 0.77

Recall that the the 'water' MUName was claimed to be gravelly / sandy texture because it
is a high energy environment...

According to Safeeq, the porosity for a stony land is 0.668

In [273]:
#index of rock outcrop = 4
df_nodup['Porosity'][4] = 0.666

#index of Mucky Peat is 19
df_nodup['Porosity'][19] = 0.77

#index of 'Water > 40 acres' is 2
df_nodup['Porosity'][2] = 0.668

In [274]:
#create a column for just the first name of the map name...
df_nodup['First_Name'] = 'nan'
for i in range(len(df_nodup)):
    if df_nodup['MUName'][i] == 'Rough broken land' or df_nodup['MUName'][i] == 'Rough mountainous land':
        df_nodup['First_Name'][i] = df_nodup['MUName'][i]
    else:
        x = df_nodup['MUName'][i].split(' ')
        df_nodup['First_Name'][i] = x[0]

# Add residual saturation, wetting front suction, and pore distirbution idnex from literature using excel

In [275]:
#sort on soil texture to make it easier and add recommended values for pore size
#distribution and residual saturation....
df_nodup.sort_values('SurfText')

#export as csv for editting in excel
df_nodup.to_csv('df_nodup_add_pordist_ressat.csv')

the values of pore distribution index and residual saturation were 
determined from the literature given in maidment 1993
EXCEPT FOR:
    pore size distribution and residual water saturation of bed rock:
        values assigned were 0.01
        

The values for wetting front suction were assigned by the traingle given
in maidment 1993...
EXCEPT FOR:
    water, rock outcrop, marsh
    - water = sand
    - rock outcrop = unknown
    - marsh = clay
        THe textures were used to assign the wetting front suction according to
        the literature
    -bedrock = 0.01
    



In [276]:
df_values_filled  = pd.read_csv('df_nodup_filledIN_values.csv')

# Once again, squeeze the data to eliminate duplicates by averaging values

In [277]:
#find the mean values grouped by the sediment names...
grouped_mean = df_values_filled.groupby('First_Name').mean()

#select the columns we care about
grouped_mean = grouped_mean[['Ksatcmhr', '15Bar_WiltingPt', '3rdBar_FieldCap','Porosity','wetting_front_suction','Pore_size_dsitribution', 'residual_water_saturation']].copy()

#round the decimals....
grouped_mean = grouped_mean.round(4)

#reset the index
grouped_mean.reset_index(inplace=True)

#show dataframe
grouped_mean

First_Name  Ksatcmhr  15Bar_WiltingPt  3rdBar_FieldCap  \
0                   Alakai    8.5173           0.1840           0.4690   
1                  Hanalei    2.9601           0.2330           0.3250   
2                Hanamaulu    3.4013           0.2313           0.3273   
3                 Hihimanu    2.4757           0.2470           0.3110   
4                    Hulua    9.6029           0.2235           0.3290   
5                 Kolokolo    2.8677           0.1505           0.2685   
6                   Koolau    0.7866           0.1670           0.2870   
7                    Marsh    7.6320           0.3600           0.5200   
8                    Pooku    3.8408           0.2220           0.3070   
9                Riverwash   33.1200           0.3300           0.4600   
10                    Rock    0.0792           0.3400           0.4200   
11       Rough broken land    1.6596           0.1590           0.2730   
12  Rough mountainous land    8.7286           0.1570           0.2680   
13               Waialeale    5.8960           0.1330           0.2990   
14                   Water    9.4628           0.3300           0.4600   

    Porosity  wetting_front_suction  Pore_size_dsitribution  \
0     0.6886                50.0000                   0.165   
1     0.5577                87.5000                   0.150   
2     0.5600                96.6667                   0.150   
3     0.5393               125.0000                   0.177   
4     0.5135                42.0000                   0.177   
5     0.5738                38.5000                   0.242   
6     0.7136                86.0000                   0.150   
7     0.7700                31.0000                   0.165   
8     0.6660               100.0000                   0.150   
9     0.4251                 2.0000                   0.694   
10    0.6660                 0.0100                   0.010   
11    0.7372                40.0000                   0.177   
12    0.6006                32.0000                   0.177   
13    0.6625                42.0000                   0.177   
14    0.6680                 4.9500                   0.694   

    residual_water_saturation  
0                       0.090  
1                       0.056  
2                       0.056  
3                       0.040  
4                       0.040  
5                       0.075  
6                       0.056  
7                       0.090  
8                       0.056  
9                       0.020  
10                      0.010  
11                      0.040  
12                      0.040  
13                      0.040  
14                      0.020

Because the riverwash nd the water soil names are right next to each other
on a map and their values are very similar, it is best to assume
the riverwash = water 

In [278]:
#indexes of the water and riverwash:
iW = 14
iR = 9

#values of water getting changed to riverwash values
values = ['Ksatcmhr','15Bar_WiltingPt','3rdBar_FieldCap','Porosity','wetting_front_suction','Pore_size_dsitribution','residual_water_saturation']

#change water values to riverwash values
for i in values:
    grouped_mean[i][iW] =  grouped_mean[i][iR]

In [279]:
grouped_mean

First_Name  Ksatcmhr  15Bar_WiltingPt  3rdBar_FieldCap  \
0                   Alakai    8.5173           0.1840           0.4690   
1                  Hanalei    2.9601           0.2330           0.3250   
2                Hanamaulu    3.4013           0.2313           0.3273   
3                 Hihimanu    2.4757           0.2470           0.3110   
4                    Hulua    9.6029           0.2235           0.3290   
5                 Kolokolo    2.8677           0.1505           0.2685   
6                   Koolau    0.7866           0.1670           0.2870   
7                    Marsh    7.6320           0.3600           0.5200   
8                    Pooku    3.8408           0.2220           0.3070   
9                Riverwash   33.1200           0.3300           0.4600   
10                    Rock    0.0792           0.3400           0.4200   
11       Rough broken land    1.6596           0.1590           0.2730   
12  Rough mountainous land    8.7286           0.1570           0.2680   
13               Waialeale    5.8960           0.1330           0.2990   
14                   Water   33.1200           0.3300           0.4600   

    Porosity  wetting_front_suction  Pore_size_dsitribution  \
0     0.6886                50.0000                   0.165   
1     0.5577                87.5000                   0.150   
2     0.5600                96.6667                   0.150   
3     0.5393               125.0000                   0.177   
4     0.5135                42.0000                   0.177   
5     0.5738                38.5000                   0.242   
6     0.7136                86.0000                   0.150   
7     0.7700                31.0000                   0.165   
8     0.6660               100.0000                   0.150   
9     0.4251                 2.0000                   0.694   
10    0.6660                 0.0100                   0.010   
11    0.7372                40.0000                   0.177   
12    0.6006                32.0000                   0.177   
13    0.6625                42.0000                   0.177   
14    0.4251                 2.0000                   0.694   

    residual_water_saturation  
0                       0.090  
1                       0.056  
2                       0.056  
3                       0.040  
4                       0.040  
5                       0.075  
6                       0.056  
7                       0.090  
8                       0.056  
9                       0.020  
10                      0.010  
11                      0.040  
12                      0.040  
13                      0.040  
14                      0.020

In [280]:
grouped_mean.to_csv('grouped_soil_inital_values.csv')

In [ ]:
#to do in arcGIS:
#reclassify MUName to First name, change water to riverwash,
#merge values from this table to the First name category